### Import

In [136]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer

import sys
sys.path.append('../scripts')

from preprocessing import clear_missing_data, visualize, delete_columnns_treshold, non_useful_columns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Collecte de données

### Chargement du dataset

In [137]:
# Je charge le dataset en ne prenant que les 100 premières lignes afin de tester les fonctions
path = "../data/dataset.csv" # Chemin du dataset 
df = pd.read_csv(path, nrows=1000, sep=',',encoding="utf-8")

In [138]:
df.shape

(1000, 207)

### Préparation des données

##### Nettoyage des colonnes

In [139]:
# Suppressions des colonnes avec 100% données manquantes
df = clear_missing_data(df)

Nombre de colonnes vides (100% de valeurs manquantes) : 73


In [ ]:
# Suppressions des colonnes avec 70% données manquantes
df = delete_columnns_treshold(df)

Les colonnes supprimées sont : Index([], dtype='object')


In [ ]:
# Suppressions des colonnes non pertinentes
df = non_useful_columns(df)

Les colonnes supprimées sont : ['url', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'brands_tags', 'last_updated_datetime', 'countries_tags', 'countries_en', 'states_tags', 'states_en', 'image_url', 'image_small_url', 'image_nutrition_url', 'image_nutrition_small_url']


##### Nettoyage des lignes

In [143]:
df

,Unnamed: 0,code,creator,product_name,quantity,brands,categories,categories_tags,categories_en,labels,...,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
0,0,54,kiliweb,Limonade artisanale a la rose,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,63,kiliweb,Tablette Tanzanie,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,2,114,kiliweb,Chocolate n 3,80 g,Jeff de Bruges,NaN,NaN,NaN,"Point Vert, Fabriqué en France",...,2415.0,44.0,28.0,30.00,27.0,NaN,7.10,0.02500,0.010000,NaN
3,3,1,inf,KOJI MISO PASTE,300g,UMAMI,"Supplements, Vegtable","en:supplements,en:vegtable","Supplements,Vegtable","No gluten, Vegetarian, No artificial flavors, ...",...,874.0,6.0,1.0,21.40,11.1,2.0,11.20,11.80000,4.720000,100.000000
4,4,105,kiliweb,Paleta gran reserva - Sierra nevada-,750ml,AdvoCare,Beverages,"en:beverages-and-beverages-preparations,en:bev...","Beverages and beverages preparations,Beverages",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,105000066,kiliweb,Herbal Tea Cinnamon,NaN,NaN,"Plant-based foods and beverages, Beverages, Ho...","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Hot ...",NaN,...,0.0,0.0,0.0,60.00,0.0,0.0,0.00,1.00000,0.400000,NaN
996,996,105000073,usda-ndb-import,"Herbal Tea, Hibiscus",NaN,Lagg's,NaN,NaN,NaN,NaN,...,1117.0,0.0,NaN,60.00,NaN,NaN,66.67,0.33782,0.135128,0.000000
997,997,105000196,usda-ndb-import,Apple & Cinnamon Tea,NaN,Lagg's,"Plant-based foods and beverages, Beverages, Ho...","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Hot ...",NaN,...,0.0,0.0,NaN,60.00,NaN,NaN,66.67,0.33782,0.135128,0.000000
998,998,105000219,usda-ndb-import,"Lagg's, green tea",NaN,Lagg's,"Plant-based foods and beverages, Beverages, Ho...","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Hot ...",NaN,...,0.0,0.0,NaN,2.67,NaN,NaN,0.00,0.00000,0.000000,0.000000
